# State Management Examples

This notebook demonstrates the `State` class, a typed key-value store with dynamic prefix views designed for pipeline state management. It maintains intermediate results and configuration across pipeline steps.

## Features Demonstrated

- **Initialization from Dictionary**: Create state with predefined data
- **Typed Access**: Runtime type validation for stored values
- **Read-only Access**: Immutable views of state data
- **Prefix Views**: Access subsets using namespace prefixes with `StateView`
- **Type Inference**: Automatic type detection from stored values

## Setup

In [1]:
import sys
import os

# Add the project root to Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [2]:
# Create state with initial data
from src.idspy.core.state import State

# Initialize with dictionary
initial_data = {
    "seed": 42,
    "train.lr": 1e-3,
    "train.epochs": 100,
    "model.type": "resnet18",
    "debug": True
}

state = State({
    "global_seed": 42,        # No prefix - global key
    "debug_mode": True,       # No prefix - global key
    "train.lr": 0.001,  # "train" prefix
    "train.epochs": 100,      # "train" prefix
    "model.architecture": "resnet",  # "model" prefix
    "eval.batch_size": 16     # "eval" prefix
})
print("Initial state:")
print(dict(state.items()))

Initial state:
{'global_seed': 42, 'debug_mode': True, 'train.lr': 0.001, 'train.epochs': 100, 'model.architecture': 'resnet', 'eval.batch_size': 16}


## Typed Access Operations

Store and retrieve values with explicit type validation.

In [3]:
# Set values with type validation
state.set("eval.batch_size", 32, int)
state.set("train.lr", 0.003, float)
state.set("model.architecture", "transformer", str)

# Get values with type assertion
batch_size = state.get("eval.batch_size", int)
lr = state.get("train.lr", float)
model = state.get("model.architecture", str)

print(f"Batch size (int): {batch_size}")
print(f"Learning rate (float): {lr}")
print(f"Model name (str): {model}")

Batch size (int): 32
Learning rate (float): 0.003
Model name (str): transformer


## Read-only Dictionary Views

Get immutable views of the state data using `as_dict()`.

In [4]:
# Get read-only view of all data
readonly_data = state.read_only_view()
print("Read-only state data:")
print(dict(readonly_data))

# Try to modify (will fail)
try:
    readonly_data["new_key"] = "value"
except TypeError as e:
    print(f"\nCannot modify read-only view: {e}")

Read-only state data:
{'global_seed': 42, 'debug_mode': True, 'train.lr': 0.003, 'train.epochs': 100, 'model.architecture': 'transformer', 'eval.batch_size': 32}

Cannot modify read-only view: 'mappingproxy' object does not support item assignment


## State Views with Prefixes

Create views for specific namespaces using `state.view(prefix)`. The `strict` parameter controls which keys are included.

In [5]:
# Create a strict view for 'train' prefix (default behavior)
train_view_strict = state.view("train")  # strict=True by default
print("Training parameters (strict view - only prefixed keys):")
print(f"Keys: {list(train_view_strict.keys())}")
print(f"Items: {dict(train_view_strict.items())}")

Training parameters (strict view - only prefixed keys):
Keys: ['lr', 'epochs']
Items: {'lr': 0.003, 'epochs': 100}


### Strict vs Non-Strict Comparison

Demonstrate the key difference between strict and non-strict views with a practical example.

In [6]:
# Create a new state to clearly demonstrate strict vs non-strict behavior

print("Demo state keys:", list(state.keys()))
print()

# Compare strict vs non-strict views for "train"
train_strict = state.view("train", strict=True)
train_mixed = state.view("train", strict=False)

print("STRICT view (train prefix only):")
print(f"  Keys: {sorted(list(train_strict.keys()))}")
print()

print("NON-STRICT view (train prefix + global keys):")
print(f"  Keys: {sorted(list(train_mixed.keys()))}")
print()

print("Explanation:")
print("- Strict view: Only 'learning_rate' and 'epochs' (from train.* keys)")
print("- Non-strict view: Adds 'global_seed' and 'debug_mode' (no prefix)")
print("- Both exclude 'model.*' and 'eval.*' keys (different prefixes)")

Demo state keys: ['global_seed', 'debug_mode', 'train.lr', 'train.epochs', 'model.architecture', 'eval.batch_size']

STRICT view (train prefix only):
  Keys: ['epochs', 'lr']

NON-STRICT view (train prefix + global keys):
  Keys: ['debug_mode', 'epochs', 'global_seed', 'lr']

Explanation:
- Strict view: Only 'learning_rate' and 'epochs' (from train.* keys)
- Non-strict view: Adds 'global_seed' and 'debug_mode' (no prefix)
- Both exclude 'model.*' and 'eval.*' keys (different prefixes)


### Modifying Through Views

Changes through views update the underlying state. Both strict and non-strict views support modifications.

In [7]:
# Demonstrate modifying through different view types
train_strict.set("lr", 0.002, float)  # This adds "train.lr"

# Check that both views see the change
print("After adding lr through strict view:")
print(f"Strict view keys: {list(train_strict.keys())}")
print(f"Non-strict view keys: {list(train_mixed.keys())}")

# Get read-only view of train parameters
train_readonly = train_mixed.as_dict()
print(f"\nTraining config (read-only): {dict(train_readonly)}")

After adding lr through strict view:
Strict view keys: ['lr', 'epochs']
Non-strict view keys: ['global_seed', 'debug_mode', 'lr', 'epochs']

Training config (read-only): {'global_seed': 42, 'debug_mode': True, 'lr': 0.002, 'epochs': 100}


Note that: In non-strict mode, when keys are equal, the view penalizes the global keys.

In [8]:
state.set("lr", 0.005, float)  # This adds "lr" (global key)
print(train_mixed.get("lr", float)) # Accesses scooped "lr" instead of global "lr" = 0.005

0.002


## Type Inference and Checking

The state automatically infers types from stored values.

In [9]:
# Add various types to state
state.set("config.layers", [64, 128, 256], list)
state.set("config.dropout", 0.5, float)
state.set("config.use_batch_norm", True, bool)

# Check inferred types
print("Inferred types:")
types_mapping = state.types()
for key, typ in types_mapping.items():
    print(f"  {key}: {typ.__name__}")

print(f"\nTotal keys: {len(list(state.keys()))}")

Inferred types:
  global_seed: int
  debug_mode: bool
  train.lr: float
  train.epochs: int
  model.architecture: str
  eval.batch_size: int
  lr: float
  config.layers: list
  config.dropout: float
  config.use_batch_norm: bool

Total keys: 10


### Type Safety in Action

Demonstrate type validation and error handling.

In [10]:
# Successful type-safe access
try:
    lr = state.get("train.lr", float)
    epochs = state.get("train.epochs", int)
    debug = state.get("debug", bool)

    print("Successful typed access:")
    print(f"  Learning rate (float): {lr}")
    print(f"  Epochs (int): {epochs}")
    print(f"  Debug mode (bool): {debug}")
except (KeyError, TypeError) as e:
    print(f"Error: {e}")

Error: "Missing key 'debug'"


### Error Handling

Demonstrate type mismatches and missing key errors.

In [11]:
# Type mismatch error
try:
    state.get("train.lr", int)  # learning_rate is float, not int
except TypeError as e:
    print(f"TypeError: {e}")

# Missing key error
try:
    state.get("nonexistent_key", str)
except KeyError as e:
    print(f"KeyError: {e}")

# Attempting to set wrong type
try:
    state.set("new_param", "should_be_int", int)
except TypeError as e:
    print(f"Set TypeError: {e}")

TypeError: Key 'train.lr' contains float, not int
KeyError: "Missing key 'nonexistent_key'"
Set TypeError: Value for 'new_param' must be int, not str
